In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [14]:
#create fully connected network
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [23]:
#create simple CNN
class CNN(nn.Module):
    def __init__(self, in_channels = 1, num_classes = 10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2)) #/2
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.fc1 = nn.Linear(16*7*7, num_classes)
        
    def forward(self, x):
        x = F.relu(self.conv1(x)) #28
        x = self.pool(x) #14
        x = F.relu(self.conv2(x)) #14
        x = self.pool(x) #7
        x = x.reshape(x.shape[0],-1)
        x = self.fc1(x)
        
        return x

In [16]:
'''
Nout = (Nin + 2p - k)/s +1

Nin = number of input features
Nout = number of output features
k = convolution kernel size
p = convolution padding size
s = convolution stride size

(28+2*1-3)/1 +1 = 28
'''

'\nNout = (Nin + 2p - k)/s +1\n\nNin = number of input features\nNout = number of output features\nk = convolution kernel size\np = convolution padding size\ns = convolution stride size\n\n(28+2*1-3)/1 +1 = 28\n'

In [24]:
#set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [25]:
#hyperparameters
in_channel = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 5

In [26]:
#load data
train_dataset = datasets.MNIST(root='mnist/',train=True, transform= transforms.ToTensor(),download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='mnist/',train=False, transform= transforms.ToTensor(),download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [27]:
#initialize network
model = CNN().to(device)

In [28]:
#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [29]:
#train network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        #get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        #forward
        scores = model(data)
        loss = criterion(scores, targets)
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        #gradient_descent or adam step
        optimizer.step()

In [30]:
#check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("checking accuracy on training data")
    else:
        print("checking accuracy on testing data")
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            
        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
        
    model.train()

In [31]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

checking accuracy on training data
Got 59120 / 60000 with accuracy 98.53
checking accuracy on testing data
Got 9830 / 10000 with accuracy 98.30
